In [1]:
from collections import defaultdict, Counter
from pathlib import Path

import pandas as pd

from consent.consistency.util import get_scan_root_dir, get_scan_dirs
from consent.data.pref_menu_scan.cookie_pref_reader import read_cookie_prefs_in_scans
from consent.data.pref_menu_scan.log_reader import read_logs_in_scans
from consent.data.pref_menu_scan.cookie_decl_reader import read_cookie_decls_in_dirs
from ooutil.df_util import read_data_files

fig_dir = Path.home() / 'local_projects/consent/paper/sp22/figures'
assert fig_dir.exists() and fig_dir.is_dir()

location = 'ie'
SCAN_ROOT_DIR = get_scan_root_dir(location)
SCAN_DIRS = get_scan_dirs(location)
SCAN_DIRS

[PosixPath('/mnt/sdd/ducbui/projects/data2/consent/2023-11-21/pref_menu_scan_0k_20k')]

In [2]:
# Need to read this to report only the sites that have been analyzed
all_complies_files = SCAN_ROOT_DIR.glob('all_complies_*.parquet')
all_complies = read_data_files(all_complies_files)
all_complies.head()

100%|██████████| 1/1 [00:00<00:00, 24.87it/s]


,name,domain,path,site,comply
0,ajs_anonymous_id,.wework.com,/,wework.com,comply
1,sa-user-id-v2,www.wework.com,/,wework.com,omit
2,ajs_user_id,.wework.com,/,wework.com,comply
3,OptanonAlertBoxClosed,.www.wework.com,/,wework.com,comply
4,user,www.wework.com,/,wework.com,comply


# Analyze cookie declarations

In [3]:
# Want to read the declarations before we combine with prefs.
cookie_decls_files = SCAN_ROOT_DIR.glob('cookie_decls_*.parquet')
cookie_decls = read_data_files(cookie_decls_files)
print(f'{len(cookie_decls):,d}')
cookie_decls.head()

100%|██████████| 1/1 [00:00<00:00,  3.73it/s]

282,716


,name,domain,duration,category_id,category,consent_mode,site,lib_name,pattern_name
0,__we_bucket_id,www.wework.com,365 days,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json
1,s_xxxx,wework.com,730 days,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json
2,ajs_anonymous_id,wework.com,365 days,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json
3,OptanonConsent,www.wework.com,365 days,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json
4,__we_request_id,www.wework.com,Session,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json


In [4]:
cookie_prefs_files = SCAN_ROOT_DIR.glob('cookie_prefs_*.parquet')
cookie_prefs = read_data_files(cookie_prefs_files)
print(f'{len(cookie_prefs):,d}')
cookie_prefs.head()

100%|██████████| 1/1 [00:00<00:00,  4.22it/s]

279,350


,name,domain,duration,category_id,category,consent_mode,site,lib_name,pattern_name,consent
0,__we_bucket_id,www.wework.com,365 days,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json,True
1,s_xxxx,wework.com,730 days,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json,True
2,ajs_anonymous_id,wework.com,365 days,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json,True
3,OptanonConsent,www.wework.com,365 days,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json,True
4,__we_request_id,www.wework.com,Session,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json,True


### Check cookie preferences only, which are the ones successfully analyzed consent

In [5]:
sites = set(all_complies.site)
cookie_prefs = cookie_prefs[cookie_prefs.site.isin(sites)]

In [16]:
cookie_prefs = cookie_prefs.drop_duplicates()
print(f"Number of extracted cookie declarations/prefs: {len(cookie_prefs):,d}")
print(f"Number of sites:", cookie_prefs.site.nunique())

Number of extracted cookie declarations/prefs: 254,790
Number of sites: 1458


## Analyze number of cookies per site

In [15]:
ncookies_per_site = cookie_prefs.groupby('site').size()
print(f"Num cookie declarations per site: {ncookies_per_site.mean():.2f} ({ncookies_per_site.std():.2f} SD), ranging {ncookies_per_site.min()} to {ncookies_per_site.max()} declarations/site")

Num cookie declarations per site: 174.75 (190.96 SD), ranging 1 to 1601 declarations/site


In [8]:
print("Sites with most cookies:")
ncookies_per_site.sort_values(ascending=False)[:15]

Sites with most cookies:


site
refinery29.com      1601
paylocity.com       1581
sharecare.com       1491
autocar.co.uk       1420
motorsport.com      1355
codecanyon.net      1289
themeforest.net     1288
motor1.com          1260
tutsplus.com        1224
bettycrocker.com    1187
gameinformer.com    1154
thepointsguy.com    1079
oregonlive.com      1030
esputnik.com        1010
pennlive.com         960
dtype: int64

In [9]:
asite = 'benetton.com'  # 'chamberlain.edu'  # 'barnesandnoble.com'
# asite = 'barnesandnoble.com'
cookie_prefs[cookie_prefs.site == asite].groupby('category_id').size()

Series([], dtype: int64)

In [10]:
ncookies_per_site = cookie_decls.groupby('site').size()
ncookies_per_site.sort_values(ascending=False)

site
jabra.com                 3200
refinery29.com            2535
gameinformer.com          2308
esputnik.com              2080
workspaceoneaccess.com    1892
                          ... 
mobfox.com                   3
petmd.com                    3
ml.com                       1
bankofamerica.com            1
bac.com                      1
Length: 1463, dtype: int64

In [11]:
# Find certain cookies
cookie_decls[cookie_decls.name == 'loc'].sample(15)

,name,domain,duration,category_id,category,consent_mode,site,lib_name,pattern_name
21668,loc,addthis.com,395 days,C0004,Marketing,inactive,prweb.com,onetrust,en.json
21274,loc,addthis.com,394 days,C0004,Targeting Cookies,inactive,turnitin.com,onetrust,en.json
119126,loc,addthis.com,396 days,C0004,Targeting Cookies,inactive,curalate.com,onetrust,en.json
185014,loc,addthis.com,396 days,C0004,Targeting Cookies,active,ada.org,onetrust,en.json
59301,loc,addthis.com,394 days,C0004,Targeting Cookies,active,truist.com,onetrust,en.json
82536,loc,addthis.com,397 days,C0007,Targeting Cookies,active,whirlpool.com,onetrust,en.json
67385,loc,.addthis.com,397 days,C0004,Targeting Cookies,inactive,bazaarvoice.com,onetrust,en.json
178652,loc,addthis.com,394 days,C0004,Third-party Cookies,inactive,subsplash.com,onetrust,en.json
189061,loc,addthis.com,394 days,C0004,Targeting cookies,inactive,rsc.org,onetrust,en.json
78488,loc,addthis.com,395 days,C0004,Targeting and advertising cookies of third par...,inactive landingpage,acer.com,onetrust,en.json


In [12]:
# Case study
# pd.options.display.max_rows = 999
cookie_decls[cookie_decls.site == 'barnesandnoble.com'].groupby('category').size()
cookie_decls[(cookie_decls.site == 'barnesandnoble.com') & (cookie_decls.category == 'Functional Cookies')]

,name,domain,duration,category_id,category,consent_mode,site,lib_name,pattern_name


In [13]:
print("Sites with least cookies:")
ncookies_per_site.sort_values()[:5]

Sites with least cookies:


site
ml.com               1
bankofamerica.com    1
bac.com              1
mobfox.com           3
petmd.com            3
dtype: int64

# [Obsolete] Analyze the individual cookie preferences

In [14]:
cookie_prefs = read_cookie_prefs_in_scans(scan_dirs)
cookie_prefs.head()

NameError: name 'scan_dirs' is not defined

In [ ]:
print("Sites with extracted category preferences but cannot extract cookie lists:", set(cat_prefs.site) - set(cookie_decls.site))

NameError: name 'cat_prefs' is not defined

In [ ]:
print(f"Number of scanned sites: {len(site_dirs):,d}")
print(f"Number of successfully extracted sites: {cookie_prefs.site.nunique()}")
print(f"Number of extracted cookie decls: {len(cookie_decls):,d}")
print(f"Number of extracted cookie decls matched with preference: {len(cookie_prefs):,d}")

Number of scanned sites: 18,010
Number of successfully extracted sites: 930
Number of extracted cookie decls: 160,906
Number of extracted cookie decls matched with preference: 160,906


In [ ]:
cookielist_sites = sorted(cookie_prefs.site.unique().tolist())
n_cookielist_sites = len(cookielist_sites)
assert cookie_prefs.site.nunique() == n_cookielist_sites
n=10
print("Sites with cookie declarations:", n_cookielist_sites, f', first {n} sites:' , cookielist_sites[:n])

Sites with cookie declarations: 930 , first 10 sites: ['10fastfingers.com', '24hourfitness.com', '24s.com', '2o7.net', '3dsystems.com', 'ab-inbev.com', 'abbvie.com', 'aber.ac.uk', 'aberdeen.com', 'abus.com']


In [ ]:
# Below is still under development for new data

In [ ]:
# Try to get norm_name to every cookie, but some missing, maybe due to UI extraction.

from consent.consistency.util import normalize_cookie_category_name

prefs, errors, warnings = read_logs_in_scans(scan_dirs)
prefs['norm_name'] = prefs.category.map(normalize_cookie_category_name)

site_cat_id_to_norm_name = {}
for row in prefs.itertuples():
    site_cat_id_to_norm_name[(row.site, row.category_id)] = row.norm_name

cookie_decls['norm_name'] = cookie_decls.apply(lambda row: site_cat_id_to_norm_name.get((row['site'], row['category_id'])), axis=1)
print("Num sites with cookie categories not shown", cookie_decls[cookie_decls.norm_name.isna()].site.nunique())
# cookie_decls[cookie_decls.norm_name.isna() & (cookie_decls.site == 'optimizely.com')]
cookie_decls

NameError: name 'read_logs_in_scans' is not defined

In [ ]:
def get_num_cookies(site, norm_name):
    group = site_prefs[ (site_prefs.site == site) & (site_prefs.norm_name == norm_name) ][['host', 'name']]
    if len(group) != len(group.drop_duplicates()):
        print(f'WARNING {site=} has duplicates per category')
        # print(group)
    return len(group.drop_duplicates())

site_ncookies_data = defaultdict(list)
for site in cookie_prefs.site.unique():
    site_ncookies_data['site'].append(site)
    for norm_name in site_prefs.norm_name.unique():
        site_ncookies_data[norm_name].append(get_num_cookies(site, norm_name))
    site_ncookies_data['all_cookies'].append(len(site_prefs[ (site_prefs.site == site) ]))
site_ncookies = pd.DataFrame(site_ncookies_data)
site_ncookies
# name_to_ncookies = {norm_name: get_num_cookies(norm_name) for norm_name in site_prefs.norm_name.unique()}

NameError: name 'site_prefs' is not defined

In [ ]:
mean_site_cookies = site_ncookies.mean().sort_values(ascending=False).head(8)
mean_site_cookies

all_cookies                155.746988
Targeting                   52.325301
Performance                 17.204819
Strictly Necessary          16.951807
Functional                  15.373494
Advertising                 12.373494
Personalize Advertising     10.867470
Measure Performance          5.060241
dtype: float64

In [ ]:
top_cats = mean_site_cookies.index.tolist()
# top_cats = cat_names.head(6)['index'].tolist()
site_ncookies[top_cats]

,all_cookies,Targeting,Performance,Strictly Necessary,Functional,Advertising,Personalize Advertising,Measure Performance
0,172,98,51,12,11,0,0,0
1,89,0,0,0,0,53,0,0
2,238,141,37,33,27,0,0,0
3,78,24,36,11,6,0,0,0
4,155,117,24,9,5,0,0,0
...,...,...,...,...,...,...,...,...
78,208,167,23,8,9,0,0,0
79,98,30,6,0,7,0,0,0
80,140,57,18,52,13,0,0,0
81,34,0,10,6,0,16,0,0


In [ ]:
std_site_cookies = site_ncookies.std()
max_site_cookies = site_ncookies.max()
min_site_cookies = site_ncookies.min()
mean_site_cookies_data = defaultdict(list)
for cat in top_cats:
    mean_site_cookies_data['cat'].append(cat)
    mean_site_cookies_data['mean'].append(site_ncookies[cat].mean())
    mean_site_cookies_data['std'].append(site_ncookies[cat].std())
    mean_site_cookies_data['min'].append(site_ncookies[cat].min())
    mean_site_cookies_data['max'].append(site_ncookies[cat].max())
pd.DataFrame(mean_site_cookies_data)

,cat,mean,std,min,max
0,all_cookies,155.746988,133.582243,11,643
1,Targeting,52.325301,81.315271,0,448
2,Performance,17.204819,20.285619,0,99
3,Strictly Necessary,16.951807,22.536639,0,147
4,Functional,15.373494,30.475784,0,150
5,Advertising,12.373494,44.990164,0,338
6,Personalize Advertising,10.867470,48.983514,0,351
7,Measure Performance,5.060241,21.277120,0,115


In [ ]:
print(f"Number of cookies in cookie lists: {len(site_prefs):,d}")

Number of cookies in cookie lists: 12,927


In [ ]:
def get_unresolved_hosts():
    unresolved_hosts = Counter()
    for _, row in site_prefs.iterrows():
        try:
            resolve_host_declaration(row, row['site'])
        except ValueError:
            unresolved_hosts[row['host']] += 1
    return unresolved_hosts
# unresolved_hosts = get_unresolved_hosts()
n_unresolves = sum(unresolved_hosts.values())
n_decls = len(site_prefs)
print(f"Number of resolved host in cookie declaration: {(1 - n_unresolves / n_decls) * 100}%")
print(f"Number of unresolved host in cookie declaration: {n_unresolves}")
unresolved_hosts

NameError: name 'unresolved_hosts' is not defined

## Analysis of Contradictory Preferences

In [ ]:
# Get information of each categories: group id, editable, set of cookies (name, host)

def find_cookies(site, cat_info):
    """Return a set of tuples (name, host)."""
    found = site_prefs[ (site_prefs.site == site) & (site_prefs['group_id'] == cat_info['group_id']) ]
    return set((cookie['name'], cookie['host']) for _, cookie in found.iterrows())

def get_cat_infos(site, categories, verbose=0):
    for category in categories:
        if verbose >= 2: print(f"{category=}")
        prev_status = category['prev_status']
        cur_status = category['cur_status']
        if prev_status in ['unavailable', 'always_active']:
            assert cur_status == prev_status
            editable = False
        else:
            editable = True
        cat_info = {'editable': editable, 'group_id': category['id'], 'name': category['name']}
        cat_info['cookies'] = find_cookies(site, cat_info)
        yield cat_info

def find_contra_cookies(cat_info1, cat_info2):
    return list(cat_info1['cookies'].intersection(cat_info2['cookies']))

def find_contras(cat_infos, verbose=0):
    contras = []
    for i in range(len(cat_infos) - 1):
        for j in range(i + 1, len(cat_infos)):
            cat_info_i, cat_info_j = cat_infos[i], cat_infos[j]
            if cat_info_i['editable'] or cat_info_j['editable']:
                contra_cookies = find_contra_cookies(cat_info_i, cat_info_j)
                if len(contra_cookies) > 0:
                    if verbose >= 2: print(cat_info_i['name'], cat_info_j['name'], contra_cookies)
                    contras.append({'cat1': cat_info_i['name'], 'cat2': cat_info_j['name'], 'contras': contra_cookies})
    return contras

def get_scan_info(site):
    found = raw_scans[site == raw_scans.domain]
    assert len(found) == 1
    return found.iloc[0].to_dict()

def get_contras(verbose=0):
    found_contras = []
    for site in site_prefs.site.unique():
        # if site != 'vmware.com':            continue
        scan_info = get_scan_info(site)
        cat_infos = list(get_cat_infos(site, scan_info['category']))
        contras = find_contras(cat_infos)
        if len(contras) > 0:
            if verbose >= 2: print(site)
            for contra in contras:
                contra['site'] = site
                found_contras.append(contra)
    return pd.DataFrame(found_contras)

contra_sites = get_contras()
contra_sites = contra_sites.explode('contras').reset_index(drop=True)
contra_sites['contra_name'] = contra_sites.contras.map(lambda contra: contra[0])
contra_sites['contra_host'] = contra_sites.contras.map(lambda contra: contra[1])
contra_sites['contra_cats'] = contra_sites.apply(lambda row: f"{normalize_name(row['cat1'])}-{normalize_name(row['cat2'])}", axis=1)
contra_sites


,cat1,cat2,contras,site,contra_name,contra_host,contra_cats
0,Strictly Necessary,Advertising,"(AWSALBCORS, communities.vmware.com)",vmware.com,AWSALBCORS,communities.vmware.com,Strictly Necessary-Advertising
1,Strictly Necessary,Advertising,"(VISITOR_BEACON, communities.vmware.com)",vmware.com,VISITOR_BEACON,communities.vmware.com,Strictly Necessary-Advertising
2,Performance,Functional,"(AMCVS_5B29123F5245AD520A490D45%40AdobeOrg, vm...",vmware.com,AMCVS_5B29123F5245AD520A490D45%40AdobeOrg,vmware.com,Performance-Functional
3,Performance,Functional,"(AMCV_5B29123F5245AD520A490D45%40AdobeOrg, vmw...",vmware.com,AMCV_5B29123F5245AD520A490D45%40AdobeOrg,vmware.com,Performance-Functional
4,Performance,Advertising,"(test, vmware.com)",vmware.com,test,vmware.com,Performance-Advertising
...,...,...,...,...,...,...,...
134,Performance Cookies,Targeting Cookies,"(_gtld, elsevier.com)",elsevier.com,_gtld,elsevier.com,Performance-Targeting
135,Targeting Cookies,Optional Cookies,"(B, yahoo.com)",glassdoor.co.in,B,yahoo.com,Targeting-Optional
136,Functional Cookies,Advertising Cookies,"(language, www.slideshare.net)",pardot.com,language,www.slideshare.net,Functional-Advertising
137,Functional Cookies,Advertising Cookies,"(lpv1, pi.pardot.com)",pardot.com,lpv1,pi.pardot.com,Functional-Advertising


In [ ]:
contra_cookies = contra_sites[['site', 'cat1', 'contra_name', 'contra_host']].drop_duplicates()
contra_cookies

,site,cat1,contra_name,contra_host
0,vmware.com,Strictly Necessary,AWSALBCORS,communities.vmware.com
1,vmware.com,Strictly Necessary,VISITOR_BEACON,communities.vmware.com
2,vmware.com,Performance,AMCVS_5B29123F5245AD520A490D45%40AdobeOrg,vmware.com
3,vmware.com,Performance,AMCV_5B29123F5245AD520A490D45%40AdobeOrg,vmware.com
4,vmware.com,Performance,test,vmware.com
...,...,...,...,...
134,elsevier.com,Performance Cookies,_gtld,elsevier.com
135,glassdoor.co.in,Targeting Cookies,B,yahoo.com
136,pardot.com,Functional Cookies,language,www.slideshare.net
137,pardot.com,Functional Cookies,lpv1,pi.pardot.com


In [ ]:
print(f"Number of contra prefs: {len(contra_cookies)}, on {contra_cookies.site.nunique()} websites.")

Number of contra prefs: 78, on 22 websites.


In [ ]:
print("Most common category pairs")
contra_sites.contra_cats.value_counts()

Most common category pairs


Strictly Necessary-Targeting                                  13
Performance-Targeting                                         13
Targeting-Social Media                                        11
Performance-Social Media                                      11
Strictly Necessary-Social Media                               10
Strictly Necessary-Performance                                10
Functional-Social Media                                       10
Functional-Targeting                                          10
Functional-Performance                                        10
Strictly Necessary-Functional                                 10
Targeting-Targeting (3rd Party)                                9
Functional-Advertising                                         6
Targeting-Targeting (1st Party)                                3
Strictly Necessary-Advertising                                 2
Operate the Site and Core Services-Personalize Advertising     2
Performance-Functional   

In [ ]:
contra_cookies.contra_name.value_counts()

GoogleAdServingTest                          31
NID                                           8
language                                      5
CONSENT                                       4
tuuid_lu                                      2
__cfduid                                      2
google_push                                   2
test                                          2
c                                             2
__cf_bm                                       2
tuuid                                         2
_dlt                                          1
AWSALBCORS                                    1
dc                                            1
lpv1                                          1
EE                                            1
B                                             1
AMCVS_5B29123F5245AD520A490D45%40AdobeOrg     1
id                                            1
VISITOR_BEACON                                1
__insp_nv                               

In [ ]:
contra_cookies[contra_cookies.contra_name == 'NID']

,site,cat1,contra_name,contra_host
14,fairmont.com,Strictly Necessary Cookies,NID,google.com
22,fairmont.com,Functional Cookies,NID,google.com
28,fairmont.com,Performance Cookies,NID,google.com
32,fairmont.com,Targeting Cookies,NID,google.com
53,accorhotels.com,Strictly Necessary Cookies,NID,google.com
85,accorhotels.com,Functional Cookies,NID,google.com
110,accorhotels.com,Performance Cookies,NID,google.com
125,accorhotels.com,Targeting Cookies,NID,google.com


In [ ]:
raw_scans[raw_scans.domain == 'vmware.com'].iloc[0]['category']

[{'id': 'C0001',
  'name': 'Strictly Necessary',
  'prev_status': 'unavailable',
  'cur_status': 'unavailable'},
 {'id': 'C0002',
  'name': 'Performance',
  'prev_status': 'true',
  'cur_status': 'false'},
 {'id': 'C0003',
  'name': 'Functional',
  'prev_status': 'true',
  'cur_status': 'false'},
 {'id': 'C0004',
  'name': 'Advertising',
  'prev_status': 'true',
  'cur_status': 'false'},
 {'id': 'C0005',
  'name': 'Social Media',
  'prev_status': 'true',
  'cur_status': 'false'}]

## Analyze preference-cookie

In [ ]:
# Read in the post-rejection cookies.
def get_brcookies():
    found_brcookies = []
    for site in scans.domain.unique():
        postrej_file = data_dir / site / 'postrej_cookies.json'
        if not postrej_file.exists():
            print(f"Warning: {postrej_file} not exist")
            continue
        for brcookie in json.loads(postrej_file.read_text())['browser_cookies']:
            brcookie['site'] = site
            found_brcookies.append(brcookie)
    return pd.DataFrame(found_brcookies)
brcookies = get_brcookies()
brcookies



,domain,expires,httpOnly,name,path,sameSite,secure,value,site
0,.siemens.com,-1.000000e+00,False,s_cc,/,None,False,true,siemens.com
1,.siemens.com,1.651627e+09,False,OptanonAlertBoxClosed,/,None,True,2021-05-04T01:24:38.671Z,siemens.com
2,.siemens.com,1.651627e+09,False,OptanonConsent,/,None,True,isIABGlobal=false&datestamp=Mon+May+03+2021+21...,siemens.com
3,assets.new.siemens.com,1.620696e+09,False,AWSALB,/,None,False,t5+th0fcH+Hbtk+jobCQjqOphUp9jgprWJuErdpItNQ7y3...,siemens.com
4,assets.new.siemens.com,1.620696e+09,False,AWSALBCORS,/,None,True,t5+th0fcH+Hbtk+jobCQjqOphUp9jgprWJuErdpItNQ7y3...,siemens.com
...,...,...,...,...,...,...,...,...,...
19562,.doubleclick.net,1.683164e+09,True,IDE,/,None,True,AHWqTUk1y_w5tZLqrd9Y-Gga9-umtHBCDAk5C2sKwckV2u...,sendinblue.com
19563,.sendinblue.com,1.651628e+09,False,OptanonAlertBoxClosed,/,Lax,False,2021-05-04T01:38:35.521Z,sendinblue.com
19564,.sendinblue.com,1.622684e+09,False,pricing_version,/,None,False,v2,sendinblue.com
19565,.sendinblue.com,1.622684e+09,False,localization,/,None,False,international,sendinblue.com


In [ ]:
pref_cookies = brcookies[brcookies.name == 'OptanonConsent']
for _, row in pref_cookies.iterrows():
    if row['domain'] != '.' + row['site']:
        print("Restrictive consent scope:", row['site'], row['domain'])

Restrictive consent scope: www.fendi.com
Restrictive consent scope: .gisday.com
Restrictive consent scope: .arcgis.com
Restrictive consent scope: .www.ikea.com
Restrictive consent scope: .mailchimp.com
Restrictive consent scope: .supersonic.com
Restrictive consent scope: .home.kpmg
Restrictive consent scope: .netflix.com
Restrictive consent scope: www.mcdonalds.com
Restrictive consent scope: .www.viber.com
Restrictive consent scope: .axs.com
Restrictive consent scope: .www.salesforce.com
Restrictive consent scope: .www.fairmont.com
Restrictive consent scope: .airship.com
Restrictive consent scope: .www.cloudflare.com
Restrictive consent scope: .www.icims.com
Restrictive consent scope: .fastly.com
Restrictive consent scope: .amobee.com
Restrictive consent scope: .www.hindawi.com
Restrictive consent scope: .www.appannie.com
Restrictive consent scope: .vdx.tv
Restrictive consent scope: .www.flaticon.com
Restrictive consent scope: .www.salesforce.com
Restrictive consent scope: .adobe.com
R